# 03_exploitation.ipynb

## 🎯 Interface utilisateur Gradio avec variables explicites

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from fpdf import FPDF
import gradio as gr

c:\Users\moussa.aouinane\Documents\Ecom\Score Churn Model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 🔹 Charger le modèle, le preprocessor et les noms des colonnes originales
MODEL_PATH = "../models/churn_xgb_model.pkl"
PREPROCESSOR_PATH = "../models/churn_preprocessor.pkl"
ORIGINAL_COLUMNS_PATH = "../models/X_original_columns.pkl" # Chemin vers les noms de colonnes sauvegardés

try:
    model = joblib.load(MODEL_PATH)
    preprocessor = joblib.load(PREPROCESSOR_PATH)
    X_original_columns = joblib.load(ORIGINAL_COLUMNS_PATH)
    print("✅ Modèle, preprocessor et noms des colonnes originales chargés avec succès.")
except FileNotFoundError as e:
    print(f"Erreur de chargement de fichier : {e}. Assurez-vous que les fichiers .pkl existent dans ../models/")
    exit()
except Exception as e:
    print(f"Erreur lors du chargement des artefacts : {e}")
    exit()

# 🧾 Valeurs possibles pour les inputs Gradio
# Ces listes doivent correspondre aux valeurs APRÈS le nettoyage manuel des catégories (cat_fix)
login_device_options = ["Computer", "Mobile"]
payment_mode_options = ["Credit Card", "Debit Card", "E-Wallet", "UPI", "Cash on Delivery"]
gender_options = ["Male", "Female"]
order_cat_options = ["Laptop & Accessory", "Mobile", "Fashion", "Grocery", "Others"] # Ordre/valeurs après nettoyage
marital_status_options = ["Single", "Married", "Divorced"]
city_tier_options_gradio = ["1", "2", "3"] # Les inputs Gradio seront des strings, convertis en int puis en category
complain_options_gradio = ["0 (Non)", "1 (Oui)"] # Entrée descriptive, on extraira 0 ou 1


✅ Modèle, preprocessor et noms des colonnes originales chargés avec succès.


In [4]:
# 🔮 Fonction principale de prédiction
def predict_churn(device, payment, gender_val, category, status, tier_str, complain_str,
                  tenure, distance, time_app, devices, satisfaction, addresses,
                  hike, coupons, orders, days, cashback):

    # 1. Extraire les valeurs numériques de complain_str
    complain_val = int(complain_str.split(" ")[0])
    tier_val = int(tier_str)

    # 2. Construction du dictionnaire avec les noms de colonnes originaux
    input_dict = {
        'PreferredLoginDevice': device,
        'PreferredPaymentMode': payment,
        'Gender': gender_val,
        'PreferedOrderCat': category, # Attention à l'orthographe ici ('Prefered' vs 'Preferred') - doit correspondre à X_original_columns
        'MaritalStatus': status,
        'CityTier': tier_val, # Sera traité comme catégoriel par le preprocessor
        'Complain': complain_val, # Sera traité comme catégoriel par le preprocessor
        'Tenure': float(tenure) if tenure is not None else np.nan, # Gérer les inputs vides/None de Gradio
        'WarehouseToHome': float(distance) if distance is not None else np.nan,
        'HourSpendOnApp': float(time_app) if time_app is not None else np.nan,
        'NumberOfDeviceRegistered': int(devices) if devices is not None else np.nan,
        'SatisfactionScore': int(satisfaction) if satisfaction is not None else np.nan,
        'NumberOfAddress': int(addresses) if addresses is not None else np.nan,
        'OrderAmountHikeFromlastYear': float(hike) if hike is not None else np.nan,
        'CouponUsed': int(coupons) if coupons is not None else np.nan,
        'OrderCount': int(orders) if orders is not None else np.nan,
        'DaySinceLastOrder': int(days) if days is not None else np.nan,
        'CashbackAmount': float(cashback) if cashback is not None else np.nan
    }

    # 3. Créer un DataFrame Pandas en utilisant X_original_columns pour assurer l'ordre correct
    df = pd.DataFrame([input_dict], columns=X_original_columns)

    # 4. Appliquer les étapes de nettoyage des catégories (avant le preprocessor)
    # Doit être identique à 01_preparation.ipynb
    cat_fix = {
        'PreferredLoginDevice': {'Phone': 'Mobile', 'Mobile Phone': 'Mobile'},
        'PreferredPaymentMode': {'CC': 'Credit Card', 'COD': 'Cash on Delivery', 'E wallet': 'E-Wallet'},
        'PreferedOrderCat': {'Mobile Phone': 'Mobile'}
    }
    for col, mapping in cat_fix.items():
        if col in df.columns:
            df[col] = df[col].replace(mapping)

    # 5. Imputation des valeurs manquantes (si Gradio permet des champs vides et qu'ils deviennent NaN)
    # La stratégie doit être la même que pour l'entraînement.
    # Le preprocessor sauvegardé devrait idéalement gérer cela s'il a été fitté sur des données après imputation
    # ou si SimpleImputer fait partie du pipeline pour chaque type de colonne.
    # Si l'imputation a été faite manuellement avant de fitter le preprocessor dans 01_preparation,
    # il faut la répliquer ici AVANT preprocessor.transform().
    # Pour cet exemple, on suppose que le preprocessor gère l'imputation si nécessaire
    # ou que les champs Gradio sont tous requis. Si ce n'est pas le cas :
    cat_cols_for_preprocessor = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender',
                                 'PreferedOrderCat', 'MaritalStatus', 'CityTier', 'Complain']
    num_cols_for_preprocessor = [col for col in X_original_columns if col not in cat_cols_for_preprocessor]

    for col in df.columns:
        if df[col].isnull().any():
            if col in num_cols_for_preprocessor:
                # Ici, il faudrait idéalement utiliser les médianes/modes sauvegardés de l'entraînement.
                # Pour simplifier ici, si une valeur est manquante pour une numérique, on met 0
                # ATTENTION: CE N'EST PAS IDÉAL, utilisez les valeurs d'imputation de l'entraînement
                df[col].fillna(0, inplace=True)
            elif col in cat_cols_for_preprocessor:
                # Pour les catégorielles, utiliser le mode (ou une catégorie par défaut)
                df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else "Unknown", inplace=True)


    # 6. Assurer les types corrects pour les colonnes catégorielles avant le preprocessor
    for col in cat_cols_for_preprocessor:
        if col in df.columns:
            df[col] = df[col].astype('category')
            # S'assurer que les catégories sont celles attendues par le OneHotEncoder
            # (handle_unknown='ignore' devrait aider, mais c'est une source d'erreur potentielle)
            # Par exemple, si le preprocessor a été entraîné avec certaines catégories pour CityTier:
            # if col == 'CityTier' and hasattr(preprocessor.named_transformers_['cat'].named_steps['encoder'], 'categories_'):
            #     known_categories = preprocessor.named_transformers_['cat'].named_steps['encoder'].categories_[cat_cols_for_preprocessor.index(col)]
            #     df[col] = pd.Categorical(df[col], categories=known_categories)


    # 7. Appliquer le preprocessor chargé
    try:
        df_processed = preprocessor.transform(df)
    except Exception as e:
        return f"Erreur lors du prétraitement : {e}. Vérifiez les types et valeurs des inputs."

    # 8. Prédiction
    try:
        proba = model.predict_proba(df_processed)[0][1]
        prediction_val = model.predict(df_processed)[0]
        prediction_label = "⚠️ Le client risque de churner" if prediction_val == 1 else "✅ Le client semble fidèle"
        return f"{prediction_label}\nProbabilité de churn : {round(proba * 100, 2)}%"
    except Exception as e:
        return f"Erreur lors de la prédiction : {e}."

# 🎛️ Interface Gradio
inputs = [
    gr.Dropdown(login_device_options, label="Appareil de Connexion Préféré"),
    gr.Dropdown(payment_mode_options, label="Mode de Paiement Préféré"),
    gr.Radio(gender_options, label="Genre"),
    gr.Dropdown(order_cat_options, label="Catégorie de Commande Préférée"),
    gr.Dropdown(marital_status_options, label="Statut marital"),
    gr.Radio(city_tier_options_gradio, label="Niveau de la Ville (Tier)"),
    gr.Radio(complain_options_gradio, label="A-t-il déposé une plainte ?"),
    gr.Number(label="Ancienneté (mois)", value=10), # Ajout de valeurs par défaut pour test
    gr.Number(label="Distance entre entrepôt et domicile (km)", value=15),
    gr.Number(label="Heures passées sur l'app (par mois en moyenne)", value=3),
    gr.Number(label="Nb d'appareils enregistrés", value=3),
    gr.Slider(1, 5, step=1, label="Score de satisfaction", value=3),
    gr.Number(label="Nb d'adresses enregistrées", value=2),
    gr.Number(label="Augmentation du montant des commandes depuis l'an dernier (%)", value=15),
    gr.Number(label="Nb de coupons utilisés (le mois dernier)", value=1),
    gr.Number(label="Nb de commandes (le mois dernier)", value=2),
    gr.Number(label="Nb de jours depuis dernière commande", value=5),
    gr.Number(label="Montant moyen de cashback (€)", value=150)
]

interface = gr.Interface(
    fn=predict_churn,
    inputs=inputs,
    outputs=gr.Textbox(label="Résultat de la prédiction"),
    title="🧠 Prédiction de Churn Client",
    description="Entrez les caractéristiques du client pour prédire la probabilité de churn."
)

# 🚀 Lancement
if __name__ == '__main__':
    interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
